In [8]:
#行列の掛け算-1

import numpy as np

a = np.array([[0,1],[2,3]])
b = np.array([[1,2],[3,4]])
c = np.dot(a,b)

print('a=\n',a)
print('b=\n',b)
print('c=\n',c)

a=
 [[0 1]
 [2 3]]
b=
 [[1 2]
 [3 4]]
c=
 [[ 3  4]
 [11 16]]


In [16]:
#行列の掛け算-2

import numpy as np

a = np.array([1,2,3,4])
b = np.array([[1,2],[3,4],[5,6],[7,8]])
c = np.dot(a,b)

print('a=\n',a)
print('b=\n',b)
print('c=\n',c)

a=
 [1 2 3 4]
b=
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]
c=
 [50 60]


In [14]:
#行列の掛け算-エラー例

import numpy as np

a = np.array([0,1,2,3])
b = np.array([[1,2,3,4],[1,2,3,4]])
c = np.dot(a,b)


ValueError: shapes (4,) and (2,4) not aligned: 4 (dim 0) != 2 (dim 0)

In [29]:
#ニューラルネットワークの試作-1

import numpy as np

#活性化関数の定義
def sigmoid(x):
    return 1 / (1 + np.exp(-x))   

#入力層（１層目）
x = np.array([1,2])
#隠れ層（２層目）
w1 = np.array([[0.1,0.3,0.5],[0.1,0.3,0.5]])
b1 = np.array([0,-0.8,-0.5])
a1 = np.dot(x,w1) + b1
z = sigmoid(a1)

#出力層（３層目）
w2 = np.array([[0.5,1],[0.5,1],[0.5,1]])
b2 = np.array([0,0])
y = np.dot(z,w2) + b2

print('x=\n',x)
print('y=\n',y)

x=
 [1 2]
y=
 [0.91524014 1.83048028]


In [96]:
#ニューラルネットワークの試作-2(出力層の改良)

import numpy as np

#softmax関数の定義
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

#活性化関数の定義
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#入力層（１層目）
x = np.array([1,2])
#隠れ層（２層目）
w1 = np.array([[0.1,0.3,0.5],[0.1,0.3,0.5]])
b1 = np.array([0,-0.8,-0.5])
a1 = np.dot(x,w1) + b1
z = sigmoid(a1)

#出力層（３層目）
w2 = np.array([[0.5,1],[0.5,1],[0.5,1]])
b2 = np.array([0,0])
a2 = np.dot(z,w2) + b2
y = softmax(a2)

print('x=\n',x)
print('y=\n',y)

x=
 [1 2]
y=
 [0.28592874 0.71407126]


In [ ]:
#softmax関数の改良（オーバーフローを回避する）
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [98]:
#mnistの手書き画像を確認する
import sys, os
#親ディレクトリのパスを追加（datasetのフォルダにアクセスするため）
sys.path.append(os.pardir)  
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

#mnistデータをダウンロードする
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, flatten=True, one_hot_label=False)

#mnist訓練データの画像(0番目)を選択
img = x_train[0]
#1次元で画像データをダウンロードしているのでリシェイプする
img = img.reshape(28, 28)

#fromarrayで配列を画像化できる
img = Image.fromarray(img)
img.show()


In [1]:
import sys, os
sys.path.append(os.pardir) 
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax

#mnistのデータを取得する関数を定義
#load_mnistの引数はそれぞれ、
#normalize:色の濃さを数値を0~1に規格化するか
#flatten:各手書き画像のデータを１次元の配列にするか
#one_hot_label:ラベルを0~9の数値ではなく,[0,1,0,0,0,0,0,0,0,0]のようなone-hotにするか
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

#ニューラルネットワークに入力データを入れて計算する関数を定義
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

#実際にOCRの計算を実行する
#x(テストデータ),t(正解ラベル)にMNISTのデータを入れる
x, t = get_data()
#networkに既にパラメータが調整されたニューラルネットワークを入れる（ローカルフォルダから取得）
network = pickle.load(open("sample_weight.pkl",'rb'))

#for文でひとつずつ計算する
#結果が正解ラベルと一致したらaccuracy_cntを足していき正答率をカウントする
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 最も確率の高い要素のインデックスを取得
    if p == t[i]:
        accuracy_cnt += 1
        
#正答率（Accuracy）を出力
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
Accuracy:0.9352


In [91]:
x, t = get_data()
network = pickle.load(open("sample_weight.pkl", 'rb'))

batch_size = 100 # バッチの数
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
